<a href="https://colab.research.google.com/github/chumingyzx/Kaggle---Multimodal-Single-Cell-Integration/blob/main/ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import glob

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/NeurIPS/Data/'

In [ ]:
!pip install tables

In [ ]:
%%time
import pandas as pd
import numpy as np
import gc


input_path = '/content/drive/MyDrive/NeurIPS/Data/'
feature_path = '/content/drive/MyDrive/NeurIPS/Data/'
model_path = '/content/drive/MyDrive/NeurIPS/model/'
sub_path = '/content/drive/MyDrive/NeurIPS/'
ensemble_path = '/content/drive/MyDrive/NeurIPS/'

In [ ]:

def zscore(x):
    x_zscore = []
    for i in range(x.shape[0]):
        x_row = x[i]
        x_row = (x_row - np.mean(x_row)) / np.std(x_row)
        x_zscore.append(x_row)
    x_std = np.array(x_zscore)
    return x_std

test_cite_preds = np.load(ensemble_path+'cite_oof.npy')

test_cite_preds = zscore(test_cite_preds)

test_cite_ensemble = test_cite_preds

test_multi_ensemble = np.load(ensemble_path+'multi_oof.npy')

In [ ]:
metadata = pd.read_csv(input_path+'metadata.csv')[['cell_id','technology']]
evaluation_ids = pd.read_csv(input_path+'evaluation_ids.csv')
evaluation_ids = evaluation_ids.merge(metadata, on=['cell_id'], how='left')

# cite
train_cite_targets = pd.read_hdf(input_path+'train_cite_targets.h5')
cite_targets = train_cite_targets.columns.values.tolist()

del train_cite_targets
gc.collect()

test_preds_cite = pd.DataFrame(test_cite_ensemble, columns=cite_targets)

test_cite_inputs_id = pd.read_feather(feature_path+'test_cite_inputs_id.feather')
test_preds_cite['cell_id'] = test_cite_inputs_id['cell_id']
test_preds_cite = test_preds_cite[test_preds_cite['cell_id'].isin(evaluation_ids['cell_id'])]
test_preds_cite = pd.melt(test_preds_cite,id_vars='cell_id')
test_preds_cite.columns = ['cell_id','gene_id','target']

del test_cite_inputs_id
gc.collect()

# multi
train_multi_targets = pd.read_hdf(input_path+'train_multi_targets.h5')
multi_targets = train_multi_targets.columns.values.tolist()

del train_multi_targets
gc.collect()

test_preds_multi = pd.DataFrame(test_multi_ensemble, columns=multi_targets)

test_multi_inputs_id = pd.read_feather(feature_path+'test_multi_inputs_id.feather')
test_preds_multi['cell_id'] = test_multi_inputs_id['cell_id']
test_preds_multi = test_preds_multi[test_preds_multi['cell_id'].isin(evaluation_ids['cell_id'])]
test_preds_multi = pd.melt(test_preds_multi,id_vars='cell_id')
test_preds_multi.columns = ['cell_id','gene_id','target']

del test_multi_inputs_id
gc.collect()

# merge
test_preds = pd.concat([test_preds_cite,test_preds_multi])
evaluation_ids = evaluation_ids.merge(test_preds, on=['cell_id','gene_id'], how='left')
evaluation_ids[['row_id','target']].to_csv(sub_path+'submission6.csv',index=False)